# Settign up

## Conexión a google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd ./drive/MyDrive/Colab Notebooks/

In [3]:
%pwd

'/content/drive/MyDrive/Colab Notebooks'

## Instalar microTC

In [5]:
%pip install git+https://github.com/felipeRmBr/microtc.git#egg=microtc

  Cloning https://github.com/felipeRmBr/microtc.git to /tmp/pip-install-8rg6lo6j/microtc
  Running command git clone -q https://github.com/felipeRmBr/microtc.git /tmp/pip-install-8rg6lo6j/microtc
  Created wheel for microtc: filename=microtc-2.2.8-cp37-none-any.whl size=60798 sha256=d0fe64e2561429154c76edb590952d87ba48dcc3054ef4c5aee29aef1e57acbc
  Stored in directory: /tmp/pip-ephem-wheel-cache-1wqcijii/wheels/73/00/4e/a4e2fc519599a41145f1740a75c80f390d4bcef72fed066f56
Successfully built microtc


## Importar módulos *_utils

In [6]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/')
from my_utils import dataset_utils
from my_utils import eval_utils
from my_utils import svc_utils
from my_utils import results_utils

import pickle

## Importar el dataset

In [7]:
X_train, Y_train = dataset_utils.importTrainDataForSVM()

# Experimentos para ajustar los hiperparámetros de los modelos de texto

# PARTE 1

En estos experimentos no concentramos en estudiar el ajuste de 8 hiperparámetros de los modelos de texto: 'num_option', 'usr_option', 'url_option', 'hashtag_option', 'delete_punc' 'delete_dup' y 'delete_diac'; para los cuatro primeros consideramos las opciones 'delete' y 'group', y para los últimos 4, las opciones 'True' y 'False'. El resultado es un conjunto con 128 modelos de texto diferentes (dado un ajuste fijo para el resto de hiperparámetros). 

Estos 128 modelos se evaluaran de forma exhaustiva en ambas tareas combinandolos con los dos tipos de kernel disponibles y usando una sola combinación de tokens para cada subconjunto de experimentos (1 subconjunto de experimentos por cada combinacion TASK-Kernel). 

En suma, se probarán 512 clasificadores diferentes y se elegiran, por cada subconjunto de experimentos, 8 configuraciones de hiperparámetros que despues se probaran con las combinaciones de tokens restantes. 

(Debe ser claro que este plan parte de la hipotesis de que el mejor ajuste para un modelo de texto no depende de la combinación de tokens que se elija, o por lo menos no de una forma importante, por lo que debe ser suficiente con hacer una busqueda exhaustiva con una de las combinaciones y despues utilizar los resultados de esa busqueda para encontrar un ajuste aceptablemente bueno para el resto de las combinaciones de tokens)

## Experimentos para la tarea A ('HS')

** Como el combo de tokens mejor evaluado es el mismo para ambos kernels podemos ejecutar los dos experimentos en un solo paso

### TextModels variations

In [ ]:
text_model_params = {'num_option':['group','delete'],
                     'usr_option':['group','delete'],
                     'url_option':['group','delete'],
                     'hashtag_option':['group','delete'],
                     'del_punc':[True, False],
                     'del_dup':[True, False],
                     'del_diac':[True, False],
                     'token_list': [[-3, -2, -1, 3, 7]],
                     'weighting':['tfidf']
                     }

text_model_search_space = list(product(*text_model_params.values()))

for entry in text_model_search_space:
  entry.append
print(len(text_model_search_space), ' combinations')

128  combinations


### SVM variations

In [ ]:
# svm_search_space_1,  for experiment with linear kernel
svm_params = {'kernel':['linear'], 'C':[2]}
svm_search_space_1 = list(product(*svm_params.values()))
print(len(svm_search_space_1), ' combination(s) in svm_search_space_1') 

# svm_search_space_2,  for experiment with sigmoid kernel
svm_params = {'kernel':['sigmoid'], 'C':[2]}
svm_search_space_2 = list(product(*svm_params.values()))
print(len(svm_search_space_2), ' combination(s) in svm_search_space_2')             

1  combination(s) in svm_search_space_1
1  combination(s) in svm_search_space_2


### Scanning

In [ ]:
_ = svc_utils.svcSpaceScanner(X_train, Y_train,
                    'HS',
                    text_model_search_space,
                    list(text_model_params.keys()),
                    svm_search_space_1,
                    list(svm_params.keys()),
                    n_folds=7,
                    metrics=['acc'],
                    partial_CV=True,
                    backup_file='./Results/SVC/phase-3/part-1/HS_linear.df')

In [ ]:
_ = svc_utils.svcSpaceScanner(X_train, Y_train,
                    'HS',
                    text_model_search_space,
                    list(text_model_params.keys()),
                    svm_search_space_2,
                    list(svm_params.keys()),
                    n_folds=7,
                    metrics=['acc'],
                    partial_CV=True,
                    backup_file='./Results/SVC/phase-3/part-1/HS_sigmoid.df')

## Experimentos para la tarea conjunta ('HTA')

### TextModels variations

In [ ]:
text_model_params = {'num_option':['group','delete'],
                     'usr_option':['group','delete'],
                     'url_option':['group','delete'],
                     'hashtag_option':['group','delete'],
                     'del_punc':[True, False],
                     'del_dup':[True, False],
                     'del_diac':[True, False],
                     'token_list': [],
                     'weighting':['tfidf']
                     }

# text_model_search_space_1,  for experiment with linear kernel
text_model_params['token_list'] = [[-3, -2, -1, 3] ]
text_model_search_space_1 = list(product(*text_model_params.values()))
print(len(text_model_search_space_1), ' combinations int text_model_search_space_1')

# text_model_search_space_2,  for experiment with sigmoid kernel
text_model_params['token_list'] = [[-3, -1, 3, 7] ]
text_model_search_space_2 = list(product(*text_model_params.values()))
print(len(text_model_search_space_2), ' combinations int text_model_search_space_2')

128  combinations int text_model_search_space_1
128  combinations int text_model_search_space_2


### SVM variations

In [ ]:
# svm_search_space_1,  for experiment with linear kernel
svm_params = {'kernel':['linear'], 'C':[2]}
svm_search_space_1 = list(product(*svm_params.values()))
print(len(svm_search_space_1), ' combination(s) in svm_search_space_1') 

# svm_search_space_2,  for experiment with sigmoid kernel
svm_params = {'kernel':['sigmoid'], 'C':[2]}
svm_search_space_2 = list(product(*svm_params.values()))
print(len(svm_search_space_2), ' combination(s) in svm_search_space_2')           

1  combinations in svm_search_space_1
1  combinations in svm_search_space_2


### Scanning

In [ ]:
# scanning for classifiers with linear kernel
_ = svc_utils.svcSpaceScanner(X_train, Y_train,
                    'HTA',
                    text_model_search_space_1,
                    list(text_model_params.keys()),
                    svm_search_space_1,
                    list(svm_params.keys()),
                    n_folds=7,
                    metrics=['acc'],
                    partial_CV=True,
                    backup_file='./Results/SVC/phase-3/part-1/HTA_linear.df')

In [ ]:
# scanning for classifiers with sigmoid kernel
_ = svc_utils.svcSpaceScanner(X_train, Y_train,
                    'HTA',
                    text_model_search_space_2,
                    list(text_model_params.keys()),
                    svm_search_space_2,
                    list(svm_params.keys()),
                    n_folds=7,
                    metrics=['acc'],
                    partial_CV=True,
                    backup_file='./Results/SVC/phase-3/part-1/HTA_sigmoid.df')

## Análisis de resultados

In [ ]:
from itertools import product

In [ ]:
bin_list = [['0','1']]*7
seven_bits_codes = list(product(*bin_list))

len(seven_bits_codes), len(six_bits_codes)

In [ ]:
from pandas import DataFrame
def computeParamsImpact(resutls_df):
  bin_list = [['0','1']]*6
  six_bits_codes = list(product(*bin_list))
  six_bits_codes = [''.join(list(c)) for c in six_bits_codes]   

  params_list = ['num_opt', 'usr_opt', 'url_opt', 'hashtag_opt', 'del_punc', 'del_dup', 'del_diac']
  results = []
  for param_idx in range(7):
    results_df['partial_bin_code'] = results_df['bin_code'].apply(lambda code: ''.join([b for i, b in enumerate(code) if i!=param_idx]))
    cummulative_delta = 0
    cummulative_abs_delta = 0
    
    max_delta = 0
    min_delta = 0
    for code in six_bits_codes:
      mask = results_df['partial_bin_code'] == code
      twin_combos_metrics = results_df.loc[mask, 'mean_val_acc'].to_list()

      twin_combos_delta = twin_combos_metrics[1] - twin_combos_metrics[0]
      cummulative_delta += twin_combos_delta
      cummulative_abs_delta += abs(twin_combos_delta) 

      max_delta = max(max_delta, twin_combos_delta)
      min_delta = min(min_delta, twin_combos_delta)

    results.append({'mean_delta':cummulative_delta/64,
                    'mean_abs_delta':cummulative_abs_delta/64,
                    'max_abs_delta':max(abs(max_delta), abs(min_delta)),
                    'max_delta':max_delta,
                    'min_delta':min_delta} 
                    )
      
    #print(params_list[param_idx], '   ', cummulative_delta_1/64, '   ',  max_delta, '   ', min_delta)
  
  return DataFrame(results, index=params_list)

### Resultados (HS-linear kernel)

In [ ]:
import pickle
with open('./Results/SVC/phase-3/part-1/HS_linear.df', 'rb') as file_handler:
  results_df = pickle.load(file_handler)

In [ ]:
results_df.sort_values(by='mean_val_acc', ascending=False, inplace=True)
results_df.head(3)

,num_option,usr_option,url_option,hashtag_option,delete_punc,delete_dup,delete_diac,token_list,kernel,mean_acc,mean_val_acc
46,group,delete,group,delete,False,False,True,"[-3, -2, -1, 3, 7]",linear,0.99965,0.774485
62,group,delete,delete,delete,False,False,True,"[-3, -2, -1, 3, 7]",linear,0.99965,0.774485
58,group,delete,delete,delete,True,False,True,"[-3, -2, -1, 3, 7]",linear,0.99965,0.774135


In [ ]:
params_dict = {'num_option':['group','delete'],
               'usr_option':['group','delete'],
               'url_option':['group','delete'],
               'hashtag_option':['group','delete'],
               'delete_punc':[True,False],
               'delete_dup':[True,False],
               'delete_diac':[True,False]
               }
for param in list(params_dict.keys()):
  print(param)
  print(results_df[param][:16].value_counts())
  print()

#### Resumen de los resultados (HS/linear-kernel)

* 'num_option': ['group']
* 'usr_option': ['delete']
* 'url_option': ['delete']
* 'hashtag_option': ['delete']
* 'delete_punc': [True]
* 'delete_dup': [False]
* 'delete_diac': [True]

In [ ]:
results_df['bin_code'] = seven_bits_codes

In [ ]:
results_df[:4]

In [ ]:
computeParamsImpact(results_df)

# the delta value measures the change observed in the mean_val_acc 
# after switching the parameter X either from 'group' to 'delete'
# or from True to False  

,mean_delta,mean_abs_delta,max_abs_delta,max_delta,min_delta
num_opt,-0.000534,0.000673,0.001867,0.000700,-0.001867
usr_opt,0.002000,0.002000,0.003208,0.003208,0.000000
url_opt,0.000000,0.000000,0.000000,0.000000,0.000000
hashtag_opt,0.001431,0.001453,0.002683,0.002683,-0.000233
del_punc,-0.001179,0.001201,0.003558,0.000350,-0.003558
del_dup,0.001555,0.001555,0.003383,0.003383,0.000000
del_diac,-0.001187,0.001223,0.003150,0.000350,-0.003150


In [ ]:
params_list = results_df.columns.tolist()[:-5]
results_by_parameter = []
for param in params_list:
  results_by_parameter.append(results_df.loc[:,[param, 'mean_val_acc']].groupby(by=[param]).aggregate(['mean', 'min', 'max']))

results_by_parameter

In [ ]:
results_by_parameter

In [ ]:
results_df.sort_values(by='mean_val_acc', ascending=False, inplace=True)
results_df.iloc[:5,:]

In [ ]:
tm_settings_list_1 = results_df.iloc[:5,:7].values.tolist()

### Resultados (HS-sigmoid kernel)

In [ ]:
import pickle
with open('./Results/SVC/phase-3/part-1/HS_sigmoid.df', 'rb') as file_handler:
  results_df = pickle.load(file_handler)

In [ ]:
results_df.sort_values(by='mean_val_acc', ascending=False, inplace=True)
results_df.head(5)

,num_option,usr_option,url_option,hashtag_option,delete_punc,delete_dup,delete_diac,token_list,kernel,mean_acc,mean_val_acc
46,group,delete,group,delete,False,False,True,"[-3, -2, -1, 3, 7]",sigmoid,0.99965,0.775710
62,group,delete,delete,delete,False,False,True,"[-3, -2, -1, 3, 7]",sigmoid,0.99965,0.775710
110,delete,delete,group,delete,False,False,True,"[-3, -2, -1, 3, 7]",sigmoid,0.99965,0.775010
126,delete,delete,delete,delete,False,False,True,"[-3, -2, -1, 3, 7]",sigmoid,0.99965,0.775010
43,group,delete,group,delete,True,False,False,"[-3, -2, -1, 3, 7]",sigmoid,0.99965,0.774485


In [ ]:
params_dict = {'num_option':['group','delete'],
               'usr_option':['group','delete'],
               'url_option':['group','delete'],
               'hashtag_option':['group','delete'],
               'delete_punc':[True,False],
               'delete_dup':[True,False],
               'delete_diac':[True,False]
               }

results_utils.plotResultsByParameter1(results_df, 
                                      params_dict=params_dict)

In [ ]:
params_dict = {'num_option':['group','delete'],
               'usr_option':['group','delete'],
               'url_option':['group','delete'],
               'hashtag_option':['group','delete'],
               'delete_punc':[True,False],
               'delete_dup':[True,False],
               'delete_diac':[True,False]
               }
for param in list(params_dict.keys()):
  print(param)
  print(results_df[param][:16].value_counts())
  print()

num_option
group     12
delete     4
Name: num_option, dtype: int64

usr_option
delete    14
group      2
Name: usr_option, dtype: int64

url_option
delete    8
group     8
Name: url_option, dtype: int64

hashtag_option
delete    14
group      2
Name: hashtag_option, dtype: int64

delete_punc
False    10
True      6
Name: delete_punc, dtype: int64

delete_dup
False    10
True      6
Name: delete_dup, dtype: int64

delete_diac
True     14
False     2
Name: delete_diac, dtype: int64



#### Resumen de los resultados (HS/sigmoid kernel)

* 'num_option': ['group']
* 'usr_option': ['delete']
* 'url_option': ['delete']
* 'hashtag_option': ['delete']
* 'delete_punc': [False]
* 'delete_dup': [False]
* 'delete_diac': [True]

In [ ]:
results_df['bin_code'] = seven_bits_codes

In [ ]:
computeParamsImpact(results_df)

# the delta value measures the change observed in the mean_val_acc 
# after switching the parameter X either from 'group' to 'delete'
# or from True to False  

,mean_delta,mean_abs_delta,max_abs_delta,max_delta,min_delta
num_opt,-0.000937,0.000948,0.001983,0.000175,-0.001983
usr_opt,0.001746,0.001779,0.003325,0.003325,-0.000525
url_opt,0.000000,0.000000,0.000000,0.000000,0.000000
hashtag_opt,0.001637,0.001637,0.003092,0.003092,0.000000
del_punc,-0.000273,0.001130,0.002625,0.002625,-0.002158
del_dup,0.001684,0.001885,0.003092,0.003092,-0.001400
del_diac,-0.001079,0.001509,0.003208,0.001691,-0.003208


In [ ]:
params_list = results_df.columns.tolist()[:-8]
results_by_parameter = []
for param in params_list:
  results_by_parameter.append(results_df.loc[:,[param, 'mean_val_acc']].groupby(by=[param]).aggregate(['mean', 'min', 'max']))


In [ ]:
results_by_parameter

[           mean_val_acc                   
                    mean       min      max
 num_option                                
 delete         0.770743  0.766610  0.77501
 group          0.771680  0.768127  0.77571,
            mean_val_acc                    
                    mean       min       max
 usr_option                                 
 delete         0.772085  0.768069  0.775710
 group          0.770338  0.766610  0.773494,
            mean_val_acc                  
                    mean      min      max
 url_option                               
 delete         0.771211  0.76661  0.77571
 group          0.771211  0.76661  0.77571,
                mean_val_acc                    
                        mean       min       max
 hashtag_option                                 
 delete             0.772030  0.768244  0.775710
 group              0.770393  0.766610  0.773727,
             mean_val_acc                    
                     mean       min       max

In [ ]:
results_df.sort_values(by='mean_val_acc', ascending=False, inplace=True)
results_df.iloc[:4,:]

,num_option,usr_option,url_option,hashtag_option,delete_punc,delete_dup,delete_diac,token_list,kernel,mean_acc,mean_val_acc
46,group,delete,group,delete,False,False,True,"[-3, -2, -1, 3, 7]",sigmoid,0.99965,0.77571
62,group,delete,delete,delete,False,False,True,"[-3, -2, -1, 3, 7]",sigmoid,0.99965,0.77571
110,delete,delete,group,delete,False,False,True,"[-3, -2, -1, 3, 7]",sigmoid,0.99965,0.77501
126,delete,delete,delete,delete,False,False,True,"[-3, -2, -1, 3, 7]",sigmoid,0.99965,0.77501


In [ ]:
tm_settings_list_2 = results_df.iloc[:8,:7].values.tolist()

### Resultados (HTA-linear kernel)

In [ ]:
import pickle
with open('./Results/SVC/phase-3/part-1/HTA_linear.df', 'rb') as file_handler:
  results_df = pickle.load(file_handler)

In [ ]:
results_df.sort_values(by='mean_val_acc', ascending=False, inplace=True)

In [ ]:
params_dict =  {'num_option':['group','delete'],
                'usr_option':['group','delete'],
                'url_option':['group','delete'],
                'hashtag_option':['group','delete'],
                'del_punc':[True, False],
                'del_dup':[True, False],
                'del_diac':[True, False]
                }

for param in list(params_dict.keys()):
  print(param)
  print(results_df[param][:16].value_counts())
  print()              

num_option
group     12
delete     4
Name: num_option, dtype: int64

usr_option
delete    14
group      2
Name: usr_option, dtype: int64

url_option
delete    11
group      5
Name: url_option, dtype: int64

hashtag_option
group     10
delete     6
Name: hashtag_option, dtype: int64

del_punc
False    13
True      3
Name: del_punc, dtype: int64

del_dup
False    9
True     7
Name: del_dup, dtype: int64

del_diac
True     10
False     6
Name: del_diac, dtype: int64



#### Resumen de los resultados (HS/sigmoid kernel)

* 'num_option': ['group']
* 'usr_option': ['delete']
* 'url_option': ['delete']
* 'hashtag_option': ['group']
* 'delete_punc': [False]
* 'delete_dup': [False, True]
* 'delete_diac': [True]

In [ ]:
results_df['bin_code'] = seven_bits_codes

In [ ]:
results_df[:4]

,num_option,usr_option,url_option,hashtag_option,del_punc,del_dup,del_diac,token_list,weighting,kernel,C,mean_acc,mean_val_acc,mean_f1,mean_val_f1
0,group,group,group,group,True,True,True,"[-3, -2, -1, 3]",tfidf,linear,2,0.9993,0.729277,0.999584,0.414143
1,group,group,group,group,True,True,False,"[-3, -2, -1, 3]",tfidf,linear,2,0.9993,0.728927,0.999584,0.412365
2,group,group,group,group,True,False,True,"[-3, -2, -1, 3]",tfidf,linear,2,0.9993,0.729510,0.999584,0.413299
3,group,group,group,group,True,False,False,"[-3, -2, -1, 3]",tfidf,linear,2,0.9993,0.728927,0.999584,0.412021


In [ ]:
computeParamsImpact(results_df)

# the delta value measures the change observed in the mean_val_acc 
# after switching the parameter X either from 'group' to 'delete'
# or from True to False  

,mean_delta,mean_abs_delta,max_abs_delta,max_delta,min_delta
num_opt,-0.000727,0.000742,0.001692,0.000233,-0.001692
usr_opt,0.000822,0.000964,0.002159,0.002159,-0.000992
url_opt,0.000361,0.000509,0.001575,0.001575,-0.001458
hashtag_opt,-0.000073,0.000372,0.000933,0.000933,-0.000817
del_punc,0.000509,0.000658,0.002217,0.002217,-0.000933
del_dup,-0.000015,0.000691,0.002100,0.001983,-0.002100
del_diac,-0.000598,0.000915,0.002275,0.001867,-0.002275


In [ ]:
params_list = results_df.columns.tolist()[:-8]
results_by_parameter = []
for param in params_list:
  #print(param)
  results_by_parameter.append(results_df.loc[:,[param, 'mean_val_acc']].groupby(by=[param]).aggregate(['mean', 'min', 'max']))


In [ ]:
results_by_parameter

[           mean_val_acc                    
                    mean       min       max
 num_option                                 
 delete         0.729089  0.726885  0.731027
 group          0.729817  0.728169  0.731669,
            mean_val_acc                    
                    mean       min       max
 usr_option                                 
 delete         0.729864  0.727994  0.731669
 group          0.729042  0.726885  0.730502,
            mean_val_acc                    
                    mean       min       max
 url_option                                 
 delete         0.729633  0.727527  0.731319
 group          0.729273  0.726885  0.731669,
                mean_val_acc                    
                        mean       min       max
 hashtag_option                                 
 delete             0.729417  0.726885  0.731319
 group              0.729489  0.727644  0.731669,
          mean_val_acc                    
                  mean       min 

In [ ]:
results_df.sort_values(by='mean_val_acc', ascending=False, inplace=True)
results_df.iloc[:4,:]

,num_option,usr_option,url_option,hashtag_option,del_punc,del_dup,del_diac,token_list,weighting,kernel,C,mean_acc,mean_val_acc,mean_f1,mean_val_f1
34,group,delete,group,group,True,False,True,"[-3, -2, -1, 3]",tfidf,linear,2,0.998951,0.731669,0.999169,0.414056
62,group,delete,delete,delete,False,False,True,"[-3, -2, -1, 3]",tfidf,linear,2,0.999300,0.731319,0.999377,0.412182
46,group,delete,group,delete,False,False,True,"[-3, -2, -1, 3]",tfidf,linear,2,0.999650,0.731144,0.999792,0.411291
42,group,delete,group,delete,True,False,True,"[-3, -2, -1, 3]",tfidf,linear,2,0.998951,0.731085,0.999169,0.412669


In [ ]:
tm_settings_list_3 = results_df.iloc[:8,:7].values.tolist()

### Resultados (HTA-sigmoid kernel)

In [ ]:
import pickle
with open('./Results/SVC/phase-3/part-1/HTA_sigmoid.df', 'rb') as file_handler:
  results_df = pickle.load(file_handler)

In [ ]:
results_df.sort_values(by='mean_val_acc', ascending=False, inplace=True)

In [ ]:
params_dict = {'num_option':['group','delete'],
               'usr_option':['group','delete'],
               'url_option':['group','delete'],
               'hashtag_option':['group','delete'],
               'delete_punc':[True,False],
               'delete_dup':[True,False],
               'delete_diac':[True,False]
               }

results_utils.plotResultsByParameter1(results_df, 
                                      params_dict=params_dict)

In [ ]:
params_dict =  {'num_option':['group','delete'],
                'usr_option':['group','delete'],
                'url_option':['group','delete'],
                'hashtag_option':['group','delete'],
                'delete_punc':[True, False],
                'delete_dup':[True, False],
                'delete_diac':[True, False]
                }

for param in list(params_dict.keys()):
  print(param)
  print(results_df[param][:16].value_counts())
  print()    

num_option
group    16
Name: num_option, dtype: int64

usr_option
delete    10
group      6
Name: usr_option, dtype: int64

url_option
group     8
delete    8
Name: url_option, dtype: int64

hashtag_option
delete    10
group      6
Name: hashtag_option, dtype: int64

delete_punc
True     10
False     6
Name: delete_punc, dtype: int64

delete_dup
False    14
True      2
Name: delete_dup, dtype: int64

delete_diac
True    16
Name: delete_diac, dtype: int64



#### Resumen de los resultados (HS/sigmoid kernel)

* 'num_option': ['group']
* 'usr_option': ['delete']
* 'url_option': ['group']
* 'hashtag_option': ['delete']
* 'delete_punc': [True]
* 'delete_dup': [False]
* 'delete_diac': [True]

#### Análisis de impacto

In [ ]:
results_df['bin_code'] = seven_bits_codes

In [ ]:
results_df[:4]

,num_option,usr_option,url_option,hashtag_option,delete_punc,delete_dup,delete_diac,token_list,kernel,mean_acc,mean_val_acc,bin_code
0,group,group,group,group,True,True,True,"[-3, -1, 3, 7]",sigmoid,0.997551,0.732194,"(0, 0, 0, 0, 0, 0, 0)"
1,group,group,group,group,True,True,False,"[-3, -1, 3, 7]",sigmoid,0.998251,0.730327,"(0, 0, 0, 0, 0, 0, 1)"
2,group,group,group,group,True,False,True,"[-3, -1, 3, 7]",sigmoid,0.997901,0.733244,"(0, 0, 0, 0, 0, 1, 0)"
3,group,group,group,group,True,False,False,"[-3, -1, 3, 7]",sigmoid,0.998251,0.732719,"(0, 0, 0, 0, 0, 1, 1)"


In [ ]:
computeParamsImpact(results_df)

# the delta value measures the change observed in the mean_val_acc 
# after switching the parameter X either from 'group' to 'delete'
# or from True to False  

,mean_delta,mean_abs_delta,max_abs_delta,max_delta,min_delta
num_opt,-0.000638,0.000766,0.001633,0.000700,-0.001633
usr_opt,0.000715,0.000846,0.001925,0.001925,-0.000583
url_opt,0.000000,0.000000,0.000000,0.000000,0.000000
hashtag_opt,-0.000226,0.000499,0.001283,0.000642,-0.001283
del_punc,-0.000642,0.000682,0.001867,0.000292,-0.001867
del_dup,0.001079,0.001108,0.002392,0.002392,-0.000408
del_diac,-0.001502,0.001506,0.002975,0.000058,-0.002975


In [ ]:
params_list = results_df.columns.tolist()[:-8]
results_by_parameter = []
for param in params_list:
  results_by_parameter.append(results_df.loc[:,[param, 'mean_val_acc']].groupby(by=[param]).aggregate(['mean', 'min', 'max']))


In [ ]:
results_by_parameter

[           mean_val_acc                    
                    mean       min       max
 num_option                                 
 delete         0.731388  0.729219  0.732894
 group          0.732026  0.729044  0.733885,
            mean_val_acc                    
                    mean       min       max
 usr_option                                 
 delete         0.732064  0.729919  0.733769
 group          0.731350  0.729044  0.733885,
            mean_val_acc                    
                    mean       min       max
 url_option                                 
 delete         0.731707  0.729044  0.733885
 group          0.731707  0.729044  0.733885,
                mean_val_acc                    
                        mean       min       max
 hashtag_option                                 
 delete             0.731594  0.729044  0.733885
 group              0.731820  0.729452  0.733419,
             mean_val_acc                    
                     mean     

In [ ]:
results_df.sort_values(by='mean_val_acc', ascending=False, inplace=True)
results_df.iloc[:4,:]

,num_option,usr_option,url_option,hashtag_option,delete_punc,delete_dup,delete_diac,token_list,kernel,mean_acc,mean_val_acc
10,group,group,group,delete,True,False,True,"[-3, -1, 3, 7]",sigmoid,0.998251,0.733885
26,group,group,delete,delete,True,False,True,"[-3, -1, 3, 7]",sigmoid,0.998251,0.733885
46,group,delete,group,delete,False,False,True,"[-3, -1, 3, 7]",sigmoid,0.998251,0.733769
62,group,delete,delete,delete,False,False,True,"[-3, -1, 3, 7]",sigmoid,0.998251,0.733769


In [ ]:
tm_settings_list_4 = results_df.iloc[:8,:7].values.tolist()

# PARTE 2

## Espacios de búsqueda

### HS

In [ ]:
# HS, linear kernel
tm_settings_list_1 = [['group', 'delete', 'group', 'delete', False, False, True, [], 'tfidf'],
['group', 'delete', 'delete', 'delete', False, False, True, [], 'tfidf'],
['group', 'delete', 'delete', 'delete', True, False, True, [], 'tfidf'],
['group', 'delete', 'group', 'delete', True, False, True, [], 'tfidf'],
['delete', 'delete', 'group', 'delete', True, False, True, [], 'tfidf']]

tokens_combos_HS_1 = [[-3, -2, -1, 3, 7],
 [-2, -1, 3, 7],
 [-2, -1, 3, 5, 7],
 [-3, -2, -1, 3, 6],
 [-3, -1, 3, 5, 7]]


# Hs, sigmoid kernel
tm_settings_list_2=[['group', 'delete', 'group', 'delete', False, False, True, [], 'tfidf'],
['group', 'delete', 'delete', 'delete', False, False, True, [], 'tfidf'],
['delete', 'delete', 'group', 'delete', False, False, True, [], 'tfidf'],
['delete', 'delete', 'delete', 'delete', False, False, True, [], 'tfidf'],
['group', 'delete', 'group', 'delete', True, False, False, [], 'tfidf']]

tokens_combos_HS_2 = [[-3, -2, -1, 3, 7],
 [-2, -1, 3, 7],
 [-3, -1, 3, 7],
 [-3, -2, -1, 3, 6],
 [-2, -1, 3, 6, 7]]

### TR

In [ ]:
# TR, linear kernel
tm_settings_list_1 = [['group', 'delete', 'group', 'delete', False, False, True, [], 'tfidf'],
['group', 'delete', 'delete', 'delete', False, False, True, [], 'tfidf'],
['group', 'delete', 'delete', 'delete', True, False, True, [], 'tfidf'],
['group', 'delete', 'group', 'delete', True, False, True, [], 'tfidf'],
['delete', 'delete', 'group', 'delete', True, False, True, [], 'tfidf']]

tokens_combos_TR_1 = [[-3, -2, -1, 3, 6],
 [-1, 3, 7],
 [-3, -1, 3, 7],
 [-2, -1, 3, 6],
 [-3, -2, -1, 3, 7]]

# TR, sigmoid kernel
tm_settings_list_2=[['group', 'delete', 'group', 'delete', False, False, True, [], 'tfidf'],
['group', 'delete', 'delete', 'delete', False, False, True, [], 'tfidf'],
['delete', 'delete', 'group', 'delete', False, False, True, [], 'tfidf'],
['delete', 'delete', 'delete', 'delete', False, False, True, [], 'tfidf'],
['group', 'delete', 'group', 'delete', True, False, False, [], 'tfidf']]

tokens_combos_TR_2 =  [[-3, -1, 3, 7],
 [-3, -2, -1, 3, 6],
 [-3, -2, -1, 3, 7],
 [-3, -1, 3, 6, 7],
 [-2, -1, 3, 5, 7]]

### AG

In [ ]:
# AG, linear kernel
tm_settings_list_1 = [['group', 'delete', 'group', 'delete', False, False, True, [], 'tfidf'],
['group', 'delete', 'delete', 'delete', False, False, True, [], 'tfidf'],
['group', 'delete', 'delete', 'delete', True, False, True, [], 'tfidf'],
['group', 'delete', 'group', 'delete', True, False, True, [], 'tfidf'],
['delete', 'delete', 'group', 'delete', True, False, True, [], 'tfidf']]

tokens_combos_AG_1 = [[-1, 3, 7], 
                      [-3, -1, 3, 7], 
                      [-2, -1, 3, 6], 
                      [-3, -1, 3, 6],
                      [-2, -1, 3, 7]]

# AG, sigmoid kernel
tm_settings_list_2=[['group', 'delete', 'group', 'delete', False, False, True, [], 'tfidf'],
['group', 'delete', 'delete', 'delete', False, False, True, [], 'tfidf'],
['delete', 'delete', 'group', 'delete', False, False, True, [], 'tfidf'],
['delete', 'delete', 'delete', 'delete', False, False, True, [], 'tfidf'],
['group', 'delete', 'group', 'delete', True, False, False, [], 'tfidf']]

tokens_combos_AG_2 =  [[-3, -1, 3, 7], 
                       [-3, -1, 3, 6], 
                       [-2, -1, 3, 6], 
                       [-1, 3, 5, 7], 
                       [-2, -1, 3, 7]]

### HTA

In [ ]:
# HTA , linear kernel
tm_settings_list_3=[['group', 'delete', 'group', 'group', True, False, True, [], 'tfidf'],
['group', 'delete', 'delete', 'delete', False, False, True, [], 'tfidf'],
['group', 'delete', 'group', 'delete', False, False, True, [], 'tfidf'],
['group', 'delete', 'group', 'delete', True, False, True, [], 'tfidf'],
['delete', 'delete', 'delete', 'group', False, True, False, [], 'tfidf']]

tokens_combos_HTA_1 = [[-3, -2, -1, 3],
 [-3, -2, -1, 3, 4],
 [-2, -1, 3],
 [-3, -1, 3],
 [-2, -1, 3, 4, 7]]


# HTA, sigmoid kernel
tm_settings_list_4=[['group', 'group', 'group', 'delete', True, False, True, [], 'tfidf'],
['group', 'group', 'delete', 'delete', True, False, True, [], 'tfidf'],
['group', 'delete', 'group', 'delete', False, False, True, [], 'tfidf'],
['group', 'delete', 'delete', 'delete', False, False, True, [], 'tfidf'],
['group', 'group', 'delete', 'delete', False, False, True, [], 'tfidf']]

tokens_combos_HTA_2 = [[-3, -1, 3, 7],
 [-1, 3, 7],
 [-3, -2, -1, 3, 6],
 [-3, -1, 3, 6],
 [-3, -2, -1, 3, 5]]


## Escaneo

In [ ]:
from microtc.textmodel import TextModel

In [ ]:
tm_parameters_keys = ['num_option',
 'usr_option',
 'url_option',
 'hashtag_option',
 'del_punc',
 'del_dup',
 'del_diac',
 'token_list',
 'weighting']
 
svm_params_keys = ['kernel', 'C']

### HS

In [ ]:
# HS linear kernel
for idx, tokens_combo in enumerate(tokens_combos_HS_1):
  for entry in tm_settings_list_1:
    entry[-2]=tokens_combo

  svm_search_space = [['linear', 2]]
  
  # scanning for classifiers with linear kernel
  _ = svcSpaceScanner(X_train, Y_train,
                      'TR',
                      tm_settings_list_1,
                      tm_parameters_keys,
                      svm_search_space,
                      svm_params_keys,
                      n_folds=7,
                      metrics=['acc'],
                      partial_CV=True,
                      backup_file=f'./Results/SVC/phase-3/part-2/partials/HS_idx_{idx}.df',
                      save_textmodels=False,
                      save_classifiers=False)


In [ ]:
# HS sigmoid kernel
for idx, tokens_combo in enumerate(tokens_combos_HS_2):
  for entry in tm_settings_list_2:
    entry[-2]=tokens_combo

  svm_search_space = [['sigmoid', 2]]
  
  # scanning for classifiers with linear kernel
  _ = svcSpaceScanner(X_train, Y_train,
                      'HS',
                      tm_settings_list_2,
                      tm_parameters_keys,
                      svm_search_space,
                      svm_params_keys,
                      n_folds=7,
                      metrics=['acc'],
                      partial_CV=True,
                      backup_file=f'./Results/SVC/phase-3/part-2/partials/HS_idx_{idx+5}.df',
                      save_textmodels=False,
                      save_classifiers=False)

### TR

In [ ]:
# TR linear kernel
for idx, tokens_combo in enumerate(tokens_combos_TR_1):
  for entry in tm_settings_list_1:
    entry[-2]=tokens_combo

  svm_search_space = [['linear', 2]]
  
  # scanning for classifiers with linear kernel
  _ = svc_utils.svcSpaceScanner(X_train, Y_train,
                      'TR',
                      tm_settings_list_1,
                      tm_parameters_keys,
                      svm_search_space,
                      svm_params_keys,
                      n_folds=7,
                      metrics=['acc'],
                      partial_CV=True,
                      backup_file=f'./Results/SVC/phase-3/part-2/partials/TR_idx_{idx}.df',
                      save_textmodels=False,
                      save_classifiers=False)


In [ ]:
# TR sigmoid kernel
for idx, tokens_combo in enumerate(tokens_combos_TR_2):
  for entry in tm_settings_list_2:
    entry[-2]=tokens_combo

  svm_search_space = [['sigmoid', 2]]
  
  # scanning for classifiers with sigmoid kernel
  _ = svc_utils.svcSpaceScanner(X_train, Y_train,
                      'TR',
                      tm_settings_list_2,
                      tm_parameters_keys,
                      svm_search_space,
                      svm_params_keys,
                      n_folds=7,
                      metrics=['acc'],
                      partial_CV=True,
                      backup_file=f'./Results/SVC/phase-3/part-2/partials/TR_idx_{idx+5}.df',
                      save_textmodels=False,
                      save_classifiers=False)

### AG

In [ ]:
# AG linear kernel
for idx, tokens_combo in enumerate(tokens_combos_AG_1):
  for entry in tm_settings_list_1:
    entry[-2]=tokens_combo

  svm_search_space = [['linear', 2]]
  
  # scanning for classifiers with linear kernel
  _ = svc_utils.svcSpaceScanner(X_train, Y_train,
                      'AG',
                      tm_settings_list_1,
                      tm_parameters_keys,
                      svm_search_space,
                      svm_params_keys,
                      n_folds=7,
                      metrics=['acc'],
                      partial_CV=True,
                      backup_file=f'./Results/SVC/phase-3/part-2/partials/AG_idx_{idx}.df',
                      save_textmodels=False,
                      save_classifiers=False)


In [ ]:
# AG sigmoid kernel
for idx, tokens_combo in enumerate(tokens_combos_AG_2):
  for entry in tm_settings_list_2:
    entry[-2]=tokens_combo

  svm_search_space = [['sigmoid', 2]]
  
  # scanning for classifiers with sigmoid kernel
  _ = svc_utils.svcSpaceScanner(X_train, Y_train,
                      'AG',
                      tm_settings_list_1,
                      tm_parameters_keys,
                      svm_search_space,
                      svm_params_keys,
                      n_folds=7,
                      metrics=['acc'],
                      partial_CV=True,
                      backup_file=f'./Results/SVC/phase-3/part-2/partials/AG_idx_{idx+5}.df',
                      save_textmodels=False,
                      save_classifiers=False)

### HTA

In [ ]:
# HTA linear kernel
for idx, tokens_combo in enumerate(tokens_combos_3):
  for entry in tm_settings_list_3:
    entry[-2]=tokens_combo

  svm_search_space = [['linear', 2]]
  
  # scanning for classifiers with linear kernel
  _ = svcSpaceScanner(X_train, Y_train,
                      'HTA',
                      tm_settings_list_3,
                      tm_parameters_keys,
                      svm_search_space,
                      svm_params_keys,
                      n_folds=7,
                      metrics=['acc'],
                      partial_CV=True,
                      backup_file=f'./Results/SVC/phase-3/part-2/partials/HTA_idx_{idx}.df',
                      save_textmodels=False,
                      save_classifiers=False)

In [ ]:
# HTA sigmoid kernel
for idx, tokens_combo in enumerate(tokens_combos_4):
  for entry in tm_settings_list_4:
    entry[-2]=tokens_combo

  svm_search_space = [['sigmoid', 2]]
  
  # scanning for classifiers with linear kernel
  _ = svcSpaceScanner(X_train, Y_train,
                      'HTA',
                      tm_settings_list_4,
                      tm_parameters_keys,
                      svm_search_space,
                      svm_params_keys,
                      n_folds=7,
                      metrics=['acc'],
                      partial_CV=True,
                      backup_file=f'./Results/SVC/phase-3/part-2/partials/HTA_idx_{idx+5}.df',
                      save_textmodels=False,
                      save_classifiers=False)

## Resultados

In [17]:
import pandas as pd

tm_parameters_keys = ['num_option',
 'usr_option',
 'url_option',
 'hashtag_option',
 'del_punc',
 'del_dup',
 'del_diac',
 'token_list',
 'weighting']

### HS

In [8]:
import pickle

In [20]:
configurations_list = []
for idx in range(10):
  with open(f'./Results/SVC/phase-3/part-2/partials/HS_idx_{idx}.df', 'rb') as file_reader:
    results_df = pickle.load(file_reader)

  ##print(results_df.sort_values(by='mean_val_acc', ascending=False).iloc[0,2:11].values)
  ##print()

  configurations_list.append(results_df.sort_values(by='mean_val_acc', ascending=False).iloc[0,2:11].to_list())

configurations_df = pd.DataFrame(configurations_list, columns=tm_parameters_keys)

configurations_df


### TR

In [ ]:
list_of_results = [f'./Results/SVC/phase-3/part-2/partials/TR_idx_{idx}.df' for idx in range(10)]

results_df = results_utils.mergeDataFrames(list_of_results, 
                                           './Results/SVC/phase-3/part-2/TR.df')

In [ ]:
import pickle
with open('./Results/SVC/phase-3/part-2/TR.df', 'rb') as file_handler:
  results_df = pickle.load(file_handler)

In [22]:
configurations_list = []
for idx in range(10):
  with open(f'./Results/SVC/phase-3/part-2/partials/TR_idx_{idx}.df', 'rb') as file_reader:
    results_df = pickle.load(file_reader)

  ##print(results_df.sort_values(by='mean_val_acc', ascending=False).iloc[0,2:11].values)
  ##print()

  configurations_list.append(results_df.sort_values(by='mean_val_acc', ascending=False).iloc[0,2:11].to_list())

configurations_df = pd.DataFrame(configurations_list, columns=tm_parameters_keys)

configurations_df

,num_option,usr_option,url_option,hashtag_option,del_punc,del_dup,del_diac,token_list,weighting
0,group,delete,group,delete,False,False,True,"[-3, -2, -1, 3, 6]",tfidf
1,group,delete,group,delete,False,False,True,"[-1, 3, 7]",tfidf
2,group,delete,group,delete,True,False,True,"[-3, -1, 3, 7]",tfidf
3,group,delete,group,delete,False,False,True,"[-2, -1, 3, 6]",tfidf
4,group,delete,group,delete,False,False,True,"[-3, -2, -1, 3, 7]",tfidf
5,group,delete,delete,delete,False,False,True,"[-3, -1, 3, 7]",tfidf
6,group,delete,group,delete,False,False,True,"[-3, -2, -1, 3, 6]",tfidf
7,group,delete,group,delete,False,False,True,"[-3, -2, -1, 3, 7]",tfidf
8,group,delete,group,delete,False,False,True,"[-3, -1, 3, 6, 7]",tfidf
9,group,delete,group,delete,False,False,True,"[-2, -1, 3, 5, 7]",tfidf


### AG

In [ ]:
import pickle

In [ ]:
list_of_results = [f'./Results/SVC/phase-3/part-2/partials/AG_idx_{idx}.df' for idx in range(10)]

results_df = results_utils.mergeDataFrames(list_of_results, 
                                           './Results/SVC/phase-3/part-2/AG.df')

In [ ]:
import pickle
with open('./Results/SVC/phase-3/part-2/AG.df', 'rb') as file_handler:
  results_df = pickle.load(file_handler)

In [23]:
configurations_list = []

for idx in range(10):
  with open(f'./Results/SVC/phase-3/part-2/partials/AG_idx_{idx}.df', 'rb') as file_reader:
    results_df = pickle.load(file_reader)

  #print(results_df.sort_values(by='mean_val_acc', ascending=False).iloc[0,2:11].values)
  #print()

  configurations_list.append(results_df.sort_values(by='mean_val_acc', ascending=False).iloc[0,2:11].to_list())

configurations_df = pd.DataFrame(configurations_list, columns=tm_parameters_keys)

configurations_df

,num_option,usr_option,url_option,hashtag_option,del_punc,del_dup,del_diac,token_list,weighting
0,group,delete,group,delete,False,False,True,"[-2, -1, 3, 7]",tfidf
1,group,delete,group,delete,False,False,True,"[-3, -1, 3, 7]",tfidf
2,group,delete,group,delete,False,False,True,"[-2, -1, 3, 6, 7]",tfidf
3,group,delete,group,delete,False,False,True,"[-1, 3, 7]",tfidf
4,group,delete,group,delete,True,False,True,"[-3, -1, 3, 6, 7]",tfidf
5,group,delete,group,delete,False,False,True,"[-3, -2, -1, 3, 7]",tfidf
6,group,delete,delete,delete,False,False,True,"[-2, -1, 3, 7]",tfidf
7,group,delete,group,delete,False,False,True,"[-2, -1, 3, 6, 7]",tfidf
8,group,delete,group,delete,True,False,False,"[-3, -1, 3, 7]",tfidf
9,group,delete,delete,delete,False,False,True,"[-3, -1, 3, 6, 7]",tfidf


### HTA

In [25]:
configurations_list = []

for idx in range(10):
  with open(f'./Results/SVC/phase-3/part-2/partials/HTA_idx_{idx}.df', 'rb') as file_reader:
    results_df = pickle.load(file_reader)

  #print(results_df.sort_values(by='mean_val_acc', ascending=False).iloc[0,2:11].values)
  #print()

  configurations_list.append(results_df.sort_values(by='mean_val_acc', ascending=False).iloc[0,2:11].to_list())

configurations_df = pd.DataFrame(configurations_list, columns=tm_parameters_keys)

configurations_df

,num_option,usr_option,url_option,hashtag_option,del_punc,del_dup,del_diac,token_list,weighting
0,group,delete,group,group,True,False,True,"[-3, -2, -1, 3]",tfidf
1,group,delete,group,delete,True,False,True,"[-3, -2, -1, 3, 4]",tfidf
2,group,delete,group,group,True,False,True,"[-2, -1, 3]",tfidf
3,group,delete,group,group,True,False,True,"[-3, -1, 3]",tfidf
4,group,delete,group,group,True,False,True,"[-2, -1, 3, 4, 7]",tfidf
5,group,delete,delete,delete,False,False,True,"[-3, -1, 3, 7]",tfidf
6,group,group,group,delete,True,False,True,"[-1, 3, 7]",tfidf
7,group,delete,delete,delete,False,False,True,"[-3, -2, -1, 3, 6]",tfidf
8,group,group,group,delete,True,False,True,"[-3, -1, 3, 6]",tfidf
9,group,delete,group,delete,False,False,True,"[-3, -2, -1, 3, 5]",tfidf
